In [1]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-06 12:40:43.224333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

labels = pd.read_csv('labels.csv')

In [ ]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-01-15 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-01-24 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-03-28 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-06-20 00:00:00_180.png', 
    'images/Line/AI&Robotics_2019-12-26 00:00:00_365.png'   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

In [ ]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [3]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_8688/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [4]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [5]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/OHLC/S&P500_2019-02-01 00:00:00_14_crop...               5   
1149  images/OHLC/DAX_2019-02-01 00:00:00_180_croppe...              90   
6531  images/Line/EmergingMarkets_2019-02-01 00:00:0...              30   
6530  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   
6500  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   

       LastPrice  FuturePrice  Label  \
0     270.059998   270.140015      1   
1149   25.042000    27.209999      1   
6531   42.759998    42.549999      0   
6530   42.759998    42.400002      0   
6500   42.759998    42.400002      0   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
1149  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
6531  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13... 2019-02-01  
6530  [[0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    6620
0    3780
Name: Label, dtype: int64


In [ ]:
'''def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        strides = (1, 3)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:]))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax')) 

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model'''

In [7]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    # Determine the number of blocks based on the input image size
    if num_blocks == 2:
        dilation_rate = (1, 1)
    elif num_blocks == 3:
        dilation_rate = (1, 2)
    else:
        dilation_rate = (1, 3)

    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Add Convolutional layer
        if i == 0:  # Apply specific strides and dilation rate only for the first layer
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 3), padding='same', 
                             input_shape=input_shape if i == 0 else None))
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 1), dilation_rate=dilation_rate, padding='same', 
                             input_shape=input_shape if i == 0 else None))
        else:
            model.add(Conv2D(filters, kernel_size=(5, 3), padding='same'))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer
        model.add(MaxPooling2D(pool_size=(2, 1)))

    # Add Flatten and Dense layers for final prediction
    dense_units = {2: 64, 3: 128, 4: 264}.get(num_blocks, 64) 
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [8]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_trades = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_trades = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i][0] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_trades.append(100*(long_return_i))
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_trades.append(100*(short_return_i))
            short_investment += 100
    
    # Calculate average long RoR
    long_avg_ror = np.mean(long_ror)
    # Sum all long trades
    long_trades = np.sum(long_trades)
    # Calculate average short RoR
    short_avg_ror = np.mean(short_ror)
    # Sum all short trades
    short_trades = np.sum(short_trades)

    return long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(3, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(4, input_shape=(120, 602, 1))
                
                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values
                # Split the data into training and validation sets in temporal order
                split_index = int(len(X) * 0.7)
                X_train, X_test = X[:split_index], X[split_index:]
                y_train, y_test = y[:split_index], y[split_index:]
                lastPrice_test = lastPrice[split_index:]
                futurePrice_test = futurePrice[split_index:]

                # Shuffle test data
                X_test, y_test, lastPrice_test, futurePrice_test = shuffle(X_test, y_test, lastPrice_test, futurePrice_test, random_state=0)
                
                # Train the model on 10 windows of data
                window_size = len(X_train)/10
                num_windows = 10

                for epoch in range(10):  # 10 epochs
                    start = 0
                    end = int(window_size * (epoch + 1))
                    X_temp = X_train[start:end]
                    y_temp = y_train[start:end]
                    split_index = int(len(X) * 0.9)
                    X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                    y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp))            
                
                # Evaluate the model on test data
                y_pred = model.predict(X_test)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)
                
                accuracy = accuracy_score(y_test, y_pred_binary)
                precision = precision_score(y_test, y_pred_binary)
                recall = recall_score(y_test, y_pred_binary)
                f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

                y_test_array = y_test.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment = get_investment_return(y_pred_binary, lastPrice_test, futurePrice_test)
                # Total investment
                total_investment = long_investment + short_investment
                total_returns = long_trades + short_trades
                
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average Long RoR: {long_avg_ror}")
                print(f"Long Investment: {long_investment}")
                print(f"Long Return: {long_trades}")
                print(f"Average Short RoR: {short_avg_ror}")
                print(f"Short Investment: {short_investment}")
                print(f"Short Return: {short_trades}")
                print(f"Total Investment: {total_investment}")
                print(f"Total Return: {total_returns}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Return': long_trades,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment,
                    'Short_Return': short_trades, 
                    'Total_Investment': total_investment,
                    'Total_Return': total_returns
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation_scores.csv'.")

In [ ]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(3, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            elif timeframe == 365:
                model = create_cnn_model(4, input_shape=(120, 602, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values
            # Split the data into training and validation sets in temporal order
            split_index = int(len(X) * 0.75)
            X_train, X_test = X[:split_index], X[split_index:]
            y_train, y_test = y[:split_index], y[split_index:]
            lastPrice_test = lastPrice[split_index:]
            futurePrice_test = futurePrice[split_index:]
            
            window_size = len(X_train)/20

            for epoch in range(10):  # 10 epochs
                start = 0
                if epoch == 0:
                    end = int(len(X_train) * 0.5)
                else:
                    end += int(window_size * epoch)
                X_temp = X_train[start:end]
                y_temp = y_train[start:end]
                split_index = int(len(X) * 0.8)
                X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                #model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp)) 
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_split=0.9)    
            
            # Evaluate the model
            y_pred = model.predict(X_test)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)
            
            accuracy = accuracy_score(y_test, y_pred_binary)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

            y_test_array = y_test.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment = get_investment_return(y_pred_binary, lastPrice_test, futurePrice_test)
            # Total investment
            total_investment = long_investment + short_investment
            total_returns = long_trades + short_trades
            
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average Long RoR: {long_avg_ror}")
            print(f"Long Investment: {long_investment}")
            print(f"Long Return: {long_trades}")
            print(f"Average Short RoR: {short_avg_ror}")
            print(f"Short Investment: {short_investment}")
            print(f"Short Return: {short_trades}")
            print(f"Total Investment: {total_investment}")
            print(f"Total Return: {total_returns}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Long_Average_RoR': long_avg_ror,
                'Long_Investment': long_investment,
                'Long_Return': long_trades,
                'Short_Average_RoR': short_avg_ror,
                'Short_Investment': short_investment,
                'Short_Return': short_trades, 
                'Total_Investment': total_investment,
                'Total_Return': total_returns
            }, ignore_index=True)

            # Save the model
            model.save(f"models/combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_model_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_model_evaluation_scores.csv'.")

In [10]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Initialize variables for weighted sums and total count
            weighted_accuracy_sum = 0
            weighted_precision_sum = 0
            weighted_recall_sum = 0
            weighted_f1_score_sum = 0
            weighted_hit_rate_sum = 0
            total_predictions = 0
            long_avg_ror = []
            long_trades = []
            long_investment = []
            short_avg_ror = []
            short_trades = []
            short_investment = []
            total_investment = []
            total_returns = []
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(3, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            elif timeframe == 365:
                model = create_cnn_model(4, input_shape=(120, 602, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values

            # Split the data into a larger initial training set and a final test set
            split_index = int(len(X) * 0.8)
            X_train_full, X_test = X[:split_index], X[split_index:]
            y_train_full, y_test = y[:split_index], y[split_index:]
            lastPrice_train_full, lastPrice_test = lastPrice[:split_index], lastPrice[split_index:]
            futurePrice_train_full, futurePrice_test = futurePrice[:split_index], futurePrice[split_index:]

            # Define the rolling window size for training within the initial training set
            window_size = len(X_train_full) / 20

            for epoch in range(10):  # 10 epochs
                if epoch == 0:
                    start = 0
                    end = int(len(X_train_full) * 0.4)
                else:
                    start = end
                    end = start + int(window_size)

                # Calculate buffer size (5% of the training set for this window)
                buffer_size = int((end - start) * 0.05)
                
                # Define end of buffer
                buffer_end = end + buffer_size

                # Ensure buffer_end does not exceed the length of the initial training set
                buffer_end = min(buffer_end, len(X_train_full))

                # Split the data into training and test sets in temporal order
                X_train_temp = X_train_full[start:end]
                y_train_temp = y_train_full[start:end]
                lastPrice_train_temp = lastPrice_train_full[start:end]
                futurePrice_train_temp = futurePrice_train_full[start:end]

                X_test_temp = X_train_full[buffer_end:] if epoch == 9 else X_train_full[buffer_end:end+int(window_size)]
                y_test_temp = y_train_full[buffer_end:] if epoch == 9 else y_train_full[buffer_end:end+int(window_size)]
                lastPrice_test_temp = lastPrice_train_full[buffer_end:] if epoch == 9 else lastPrice_train_full[buffer_end:end+int(window_size)]
                futurePrice_test_temp = futurePrice_train_full[buffer_end:] if epoch == 9 else futurePrice_train_full[buffer_end:end+int(window_size)]

                #model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp)) 
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_split=0.1)    
            
                # Evaluate the model
                y_pred = model.predict(X_test_temp)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                num_predictions = len(y_test_temp)
                
                # Calculate the performance metrics for the current epoch
                current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                current_precision = precision_score(y_test_temp, y_pred_binary)
                current_recall = recall_score(y_test_temp, y_pred_binary)
                current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                current_hit_rate = (correct_predictions / num_predictions)

                # Update the weighted sums and total count
                weighted_accuracy_sum += current_accuracy * num_predictions
                weighted_precision_sum += current_precision * num_predictions
                weighted_recall_sum += current_recall * num_predictions
                weighted_f1_score_sum += current_f1_score * num_predictions
                total_predictions += num_predictions

                long_avg_ror_temp, long_trades_temp, long_investment_temp, short_avg_ror_temp, short_trades_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                # Total investment
                long_avg_ror.append(long_avg_ror_temp)
                long_trades.append(long_trades_temp)
                long_investment.append(long_investment_temp)
                short_avg_ror.append(short_avg_ror_temp)
                short_trades.append(short_trades_temp)
                short_investment.append(short_investment_temp)
            
            # replace nan values with 0
            long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
            long_trades = [0 if np.isnan(x) else x for x in long_trades]
            long_investment = [0 if np.isnan(x) else x for x in long_investment]
            short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
            short_trades = [0 if np.isnan(x) else x for x in short_trades]
            short_investment = [0 if np.isnan(x) else x for x in short_investment]

            # Calculate the weighted averages after the epoch loop
            weighted_average_accuracy = weighted_accuracy_sum / total_predictions
            weighted_average_precision = weighted_precision_sum / total_predictions
            weighted_average_recall = weighted_recall_sum / total_predictions
            weighted_average_f1_score = weighted_f1_score_sum / total_predictions
            weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                        
            print("Evaluation Metrics:")
            print(f"Accuracy: {weighted_average_accuracy}")
            print(f"Precision: {weighted_average_precision}")
            print(f"Recall: {weighted_average_recall}")
            print(f"F1 Score: {weighted_average_f1_score}")
            print(f"Hit Rate: {weighted_average_hit_rate}")
            print(f"Average Long RoR: {np.mean(long_avg_ror)}")
            print(f"Long Investment: {np.mean(long_investment)}")
            print(f"Long Return: {np.mean(long_trades)}")
            print(f"Average Short RoR: {np.mean(short_avg_ror)}")
            print(f"Short Investment: {np.mean(short_investment)}")
            print(f"Short Return: {np.mean(short_trades)}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': weighted_average_accuracy,
                'Precision': weighted_average_precision,
                'Recall': weighted_average_recall,
                'F1_Score': weighted_average_f1_score,
                'Hit_Rate': weighted_average_hit_rate,
                'Long_Average_RoR': np.mean(long_avg_ror),
                'Long_Investment': np.mean(long_investment),
                'Long_Return': np.mean(long_trades),
                'Short_Average_RoR': np.mean(short_avg_ror),
                'Short_Investment': np.mean(short_investment),
                'Short_Return': np.mean(short_trades)
            }, ignore_index=True)

            # Save the model
            model.save(f"models/combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_model_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_model_evaluation_scores.csv'.")

Predicting 5 days ahead with 14 days timeframe.
4/4 [==============================] - 1s 119ms/step
Evaluation Metrics:
Accuracy: 0.7208237986270023
Precision: 0.729448838298619
Recall: 0.7420081378679767
F1 Score: 0.6610602308004682
Hit Rate: 0.0
Average Long RoR: 0.007872782717975207
Long Investment: 2240.0
Long Return: 26.144312033650046
Average Short RoR: 0.004417660606942838
Short Investment: 2130.0
Short Return: 18.123467104911335
Model saved as combined_14_5.h5
--------------------------------------------------
Predicting 5 days ahead with 30 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 400ms/step


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2/2 [==============================] - 0s 66ms/step


...

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


4/4 [==============================] - 10s 2s/step
Evaluation Metrics:
Accuracy: 0.6132315521628499
Precision: 0.6132315521628499
Recall: 0.916030534351145
F1 Score: 0.6969237954553477
Hit Rate: 0.0
Average Long RoR: 0.031184058891381712
Long Investment: 3930.0
Long Return: 51.57736725949634
Average Short RoR: 0.0
Short Investment: 0.0
Short Return: 0.0


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as combined_180_90.h5
--------------------------------------------------
Predicting 5 days ahead with 365 days timeframe.
1/7 [===>..........................] - ETA: 5:33 - loss: 0.6970 - accuracy: 0.4062

KeyboardInterrupt: 

In [ ]:
ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)
                split_index = int(len(data) * 0.7)
                lastPrice = data['LastPrice'][split_index:]
                futurePrice = data['FuturePrice'][split_index:]
                # Calculate Rate of Return RoR
                ror = np.array((futurePrice - lastPrice)/lastPrice)
                # Calculate average RoR
                avg_ror = np.mean(ror)
                print(f"Average RoR for {img_type} {timeframe} {prediction}: {avg_ror}")
                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)

In [ ]:
ror_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:

            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)
            split_index = int(len(data) * 0.7)
            lastPrice = data['LastPrice'][split_index:]
            futurePrice = data['FuturePrice'][split_index:]
            # Calculate Rate of Return RoR
            ror = np.array((futurePrice - lastPrice)/lastPrice)
            # Calculate average RoR
            avg_ror = np.mean(ror)
            print(f"Average RoR for {timeframe} {prediction}: {avg_ror}")
            # Add the evaluation metrics to the DataFrame
            ror_df = ror_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Average_RoR': avg_ror
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)